In [112]:
import pandas as pd
import numpy as np
from struct import unpack
from base64 import b64decode
import string

In [113]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, concatenate, BatchNormalization, Multiply, Add
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical

In [114]:
IMG_LEN = 1024
TXT_LEN = 300
N_CLASSES = 50

In [115]:
filename = "/Users/dmitry/Downloads/topics_dataset.json"
df = pd.read_json(filename, lines=True)

In [116]:
from functools import partial

def unpck(l, x):
    return unpack('%df' % l, b64decode(x.encode('utf-8')))

unpck_img = partial(unpck, IMG_LEN)
unpck_txt = partial(unpck, TXT_LEN)

In [117]:
df_q = df.sample(frac=0.25)
x_img_q = np.stack(df_q['x1'].map(unpck_img), axis=0)
x_txt_q = np.stack(df_q['x2'].map(unpck_txt), axis=0)
y_q = to_categorical(np.array(df_q['y1']), N_CLASSES)

In [118]:
from sklearn.preprocessing import MinMaxScaler, normalize

norm_x_img_q = normalize(x_img_q)
# img_scaler = MinMaxScaler()
# img_scaler.fit(norm_x_img_q)
# scaled_x_img_q = img_scaler.transform(norm_x_img_q)

norm_x_txt_q = normalize(x_txt_q)
# txt_scaler = MinMaxScaler()
# txt_scaler.fit(norm_x_txt_q)
# scaled_x_txt_q = txt_scaler.transform(norm_x_txt_q)


In [119]:
from sklearn.model_selection import train_test_split
x_img_train, x_img_test, x_txt_train, x_txt_test, y_train, y_test = train_test_split(norm_x_img_q, norm_x_txt_q, y_q, test_size=0.2, random_state=42)

In [120]:
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.001, patience=3)

def get_model_1():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(64, activation='relu')(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(64, activation='relu')(x_img)

    x_txt = Dense(64, activation='relu')(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(64, activation='relu')(x_txt)

    x = concatenate([x_img, x_txt])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [121]:
model_1 = get_model_1()
model_1.fit([x_img_train, x_txt_train], y_train, epochs=10, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/10
38221/38221 [==============================] - 8s 201us/sample - loss: 2.4772 - accuracy: 0.3424 - val_loss: 1.8027 - val_accuracy: 0.4996
Epoch 2/10
38221/38221 [==============================] - 5s 129us/sample - loss: 1.8880 - accuracy: 0.4850 - val_loss: 1.6310 - val_accuracy: 0.5411
Epoch 3/10
38221/38221 [==============================] - 5s 134us/sample - loss: 1.7716 - accuracy: 0.5190 - val_loss: 1.5922 - val_accuracy: 0.5595
Epoch 4/10
38221/38221 [==============================] - 5s 120us/sample - loss: 1.6992 - accuracy: 0.5394 - val_loss: 1.5362 - val_accuracy: 0.5757
Epoch 5/10
38221/38221 [==============================] - 5s 118us/sample - loss: 1.6483 - accuracy: 0.5549 - val_loss: 1.5240 - val_accuracy: 0.5844
Epoch 6/10
38221/38221 [==============================] - 5s 125us/sample - loss: 1.6074 - accuracy: 0.5646 - val_loss: 1.4939 - val_accuracy: 0.5863
Epoch 7/10
38221/38221 [===========================

In [143]:
def get_model_mult():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(64, activation='relu')(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(64, activation='relu')(x_img)

    x_txt = Dense(64, activation='relu')(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(64, activation='relu')(x_txt)

    x = Multiply()([x_img, x_txt])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [146]:
model_mult = get_model_mult()
model_mult.fit([x_img_train, x_txt_train], y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 7s 173us/sample - loss: 2.6497 - accuracy: 0.2971 - val_loss: 1.8960 - val_accuracy: 0.4660
Epoch 2/20
38221/38221 [==============================] - 4s 108us/sample - loss: 1.9808 - accuracy: 0.4683 - val_loss: 1.7202 - val_accuracy: 0.5270
Epoch 3/20
38221/38221 [==============================] - 4s 95us/sample - loss: 1.8172 - accuracy: 0.5096 - val_loss: 1.6123 - val_accuracy: 0.5543
Epoch 4/20
38221/38221 [==============================] - 4s 94us/sample - loss: 1.7257 - accuracy: 0.5339 - val_loss: 1.5684 - val_accuracy: 0.5719
Epoch 5/20
38221/38221 [==============================] - 3s 91us/sample - loss: 1.6683 - accuracy: 0.5500 - val_loss: 1.5268 - val_accuracy: 0.5844
Epoch 6/20
38221/38221 [==============================] - 5s 124us/sample - loss: 1.6265 - accuracy: 0.5557 - val_loss: 1.5311 - val_accuracy: 0.5809
Epoch 7/20
38221/38221 [==============================

In [147]:
def get_model_add():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(64, activation='relu')(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(64, activation='relu')(x_img)

    x_txt = Dense(64, activation='relu')(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(64, activation='relu')(x_txt)

    x = Add()([x_img, x_txt])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [148]:
model_add = get_model_add()
model_add.fit([x_img_train, x_txt_train], y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 6s 160us/sample - loss: 2.4678 - accuracy: 0.3417 - val_loss: 1.8103 - val_accuracy: 0.4881
Epoch 2/20
38221/38221 [==============================] - 5s 140us/sample - loss: 1.9026 - accuracy: 0.4805 - val_loss: 1.6574 - val_accuracy: 0.5460
Epoch 3/20
38221/38221 [==============================] - 6s 146us/sample - loss: 1.7742 - accuracy: 0.5182 - val_loss: 1.6143 - val_accuracy: 0.5552
Epoch 4/20
38221/38221 [==============================] - 6s 146us/sample - loss: 1.6981 - accuracy: 0.5393 - val_loss: 1.5620 - val_accuracy: 0.5724
Epoch 5/20
38221/38221 [==============================] - 6s 164us/sample - loss: 1.6506 - accuracy: 0.5477 - val_loss: 1.5281 - val_accuracy: 0.5846
Epoch 6/20
38221/38221 [==============================] - 6s 161us/sample - loss: 1.6117 - accuracy: 0.5635 - val_loss: 1.5159 - val_accuracy: 0.5849
Epoch 7/20
38221/38221 [===========================

In [150]:
def get_model_mix():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(64, activation='relu')(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(64, activation='relu')(x_img)

    x_txt = Dense(64, activation='relu')(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(64, activation='relu')(x_txt)

    add = Add()([x_img, x_txt])
    add = Dense(128, activation='relu')(add)
    add = Dropout(0.25)(add)
    
    mult = Multiply()([x_img, x_txt])
    mult = Dense(128, activation='relu')(mult)
    mult = Dropout(0.25)(mult)
    
    mix = concatenate([add, mult])
    mix = Dense(128, activation='relu')(mix)
    mix = Dropout(0.25)(mix)

    out = Dense(50, activation='softmax')(mix)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [152]:
def get_model_wide_mix():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(512, activation='relu')(x_img)

    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(512, activation='relu')(x_txt)

    add = Add()([x_img, x_txt])
    add = Dense(512, activation='relu')(add)
    add = Dropout(0.25)(add)
    
    mult = Multiply()([x_img, x_txt])
    mult = Dense(512, activation='relu')(mult)
    mult = Dropout(0.25)(mult)
    
    mix = concatenate([add, mult])
    mix = Dense(512, activation='relu')(mix)
    mix = Dropout(0.25)(mix)

    out = Dense(50, activation='softmax')(mix)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [151]:
model_mix = get_model_mix()
model_mix.fit([x_img_train, x_txt_train], y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 8s 203us/sample - loss: 2.5591 - accuracy: 0.3246 - val_loss: 1.8182 - val_accuracy: 0.5046
Epoch 2/20
38221/38221 [==============================] - 6s 149us/sample - loss: 1.9552 - accuracy: 0.4730 - val_loss: 1.7078 - val_accuracy: 0.5333
Epoch 3/20
38221/38221 [==============================] - 6s 154us/sample - loss: 1.8238 - accuracy: 0.5088 - val_loss: 1.5978 - val_accuracy: 0.5663
Epoch 4/20
38221/38221 [==============================] - 5s 136us/sample - loss: 1.7495 - accuracy: 0.5280 - val_loss: 1.5621 - val_accuracy: 0.5764
Epoch 5/20
38221/38221 [==============================] - 4s 105us/sample - loss: 1.6896 - accuracy: 0.5443 - val_loss: 1.5329 - val_accuracy: 0.5738
Epoch 6/20
38221/38221 [==============================] - 4s 102us/sample - loss: 1.6471 - accuracy: 0.5526 - val_loss: 1.5321 - val_accuracy: 0.5828
Epoch 7/20
38221/38221 [===========================

In [153]:
model_wide_mix = get_model_wide_mix()
model_wide_mix.fit([x_img_train, x_txt_train], y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 42s 1ms/sample - loss: 2.1200 - accuracy: 0.4392 - val_loss: 1.5999 - val_accuracy: 0.5668
Epoch 2/20
38221/38221 [==============================] - 44s 1ms/sample - loss: 1.6445 - accuracy: 0.5581 - val_loss: 1.5601 - val_accuracy: 0.5764
Epoch 3/20
38221/38221 [==============================] - 47s 1ms/sample - loss: 1.5183 - accuracy: 0.5899 - val_loss: 1.5041 - val_accuracy: 0.5938
Epoch 4/20
38221/38221 [==============================] - 42s 1ms/sample - loss: 1.4271 - accuracy: 0.6100 - val_loss: 1.4988 - val_accuracy: 0.5919
Epoch 5/20
38221/38221 [==============================] - 49s 1ms/sample - loss: 1.3528 - accuracy: 0.6261 - val_loss: 1.4917 - val_accuracy: 0.5978
Epoch 6/20
38221/38221 [==============================] - 47s 1ms/sample - loss: 1.2845 - accuracy: 0.6426 - val_loss: 1.4658 - val_accuracy: 0.6124
Epoch 7/20
38221/38221 [==============================] -

KeyboardInterrupt: 

In [156]:
def get_model_long_horns():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(512, activation='relu')(x_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(512)(x_img)

    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(512, activation='relu')(x_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(512, activation='relu')(x_txt)

    add = Add()([x_img, x_txt])
    add = Dense(512, activation='relu')(add)
    add = Dropout(0.25)(add)
    
    mult = Multiply()([x_img, x_txt])
    mult = Dense(512, activation='relu')(mult)
    mult = Dropout(0.25)(mult)
    
    mix = concatenate([add, mult])
    mix = Dense(512, activation='relu')(mix)
    mix = Dropout(0.25)(mix)

    out = Dense(50, activation='softmax')(mix)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [157]:
model_long_horns = get_model_long_horns()
model_long_horns.fit([x_img_train, x_txt_train], y_train, epochs=8, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/8
38221/38221 [==============================] - 54s 1ms/sample - loss: 2.2581 - accuracy: 0.4020 - val_loss: 1.7444 - val_accuracy: 0.5220
Epoch 2/8
38221/38221 [==============================] - 55s 1ms/sample - loss: 1.7656 - accuracy: 0.5280 - val_loss: 1.6168 - val_accuracy: 0.5675
Epoch 3/8
38221/38221 [==============================] - 59s 2ms/sample - loss: 1.6464 - accuracy: 0.5600 - val_loss: 1.5306 - val_accuracy: 0.5905
Epoch 4/8
38221/38221 [==============================] - 61s 2ms/sample - loss: 1.5743 - accuracy: 0.5776 - val_loss: 1.5267 - val_accuracy: 0.5948
Epoch 5/8
38221/38221 [==============================] - 58s 2ms/sample - loss: 1.5105 - accuracy: 0.5935 - val_loss: 1.5469 - val_accuracy: 0.5854
Epoch 6/8
38221/38221 [==============================] - 49s 1ms/sample - loss: 1.4546 - accuracy: 0.6060 - val_loss: 1.5285 - val_accuracy: 0.5976
Epoch 7/8
38221/38221 [==============================] - 48s 1m

In [158]:
def get_model_highway_horns():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Highway()(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(512)(x_img)

    x_txt = Highway()(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(512, activation='relu')(x_txt)

    add = Add()([x_img, x_txt])
    add = Dense(512, activation='relu')(add)
    add = Dropout(0.25)(add)
    
    mult = Multiply()([x_img, x_txt])
    mult = Dense(512, activation='relu')(mult)
    mult = Dropout(0.25)(mult)
    
    mix = concatenate([add, mult])
    mix = Dense(512, activation='relu')(mix)
    mix = Dropout(0.25)(mix)

    out = Dense(50, activation='softmax')(mix)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [160]:
model_highway_horns = get_model_highway_horns()
model_highway_horns.fit([x_img_train, x_txt_train], y_train, epochs=10, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/10
38221/38221 [==============================] - 59s 2ms/sample - loss: 2.0573 - accuracy: 0.4507 - val_loss: 1.6421 - val_accuracy: 0.5449
Epoch 2/10
38221/38221 [==============================] - 48s 1ms/sample - loss: 1.6481 - accuracy: 0.5581 - val_loss: 1.5112 - val_accuracy: 0.5849
Epoch 3/10
38221/38221 [==============================] - 48s 1ms/sample - loss: 1.5368 - accuracy: 0.5853 - val_loss: 1.5427 - val_accuracy: 0.5879
Epoch 4/10
38221/38221 [==============================] - 48s 1ms/sample - loss: 1.4504 - accuracy: 0.6040 - val_loss: 1.4784 - val_accuracy: 0.6007
Epoch 5/10
38221/38221 [==============================] - 49s 1ms/sample - loss: 1.3722 - accuracy: 0.6193 - val_loss: 1.4692 - val_accuracy: 0.6082
Epoch 6/10
38221/38221 [==============================] - 52s 1ms/sample - loss: 1.2939 - accuracy: 0.6339 - val_loss: 1.4904 - val_accuracy: 0.6054
Epoch 7/10
38221/38221 [==============================] -

In [141]:
def get_model_img_1():
    inp = Input(shape=(IMG_LEN,))
    
    x = Dense(1024, activation='relu')(inp)
    x = Dropout(0.25)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [142]:
model_img_1 = get_model_img_1()
model_img_1.fit(x_img_train, y_train, epochs=10, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/10
38221/38221 [==============================] - 31s 801us/sample - loss: 2.7206 - accuracy: 0.2842 - val_loss: 2.3245 - val_accuracy: 0.3767
Epoch 2/10
38221/38221 [==============================] - 24s 623us/sample - loss: 2.3772 - accuracy: 0.3690 - val_loss: 2.2492 - val_accuracy: 0.3946
Epoch 3/10
38221/38221 [==============================] - 24s 639us/sample - loss: 2.2930 - accuracy: 0.3920 - val_loss: 2.2280 - val_accuracy: 0.4062
Epoch 4/10
38221/38221 [==============================] - 27s 716us/sample - loss: 2.2306 - accuracy: 0.4025 - val_loss: 2.1808 - val_accuracy: 0.4191
Epoch 5/10
38221/38221 [==============================] - 27s 697us/sample - loss: 2.1839 - accuracy: 0.4169 - val_loss: 2.1634 - val_accuracy: 0.4137
Epoch 6/10
38221/38221 [==============================] - 27s 709us/sample - loss: 2.1499 - accuracy: 0.4231 - val_loss: 2.1463 - val_accuracy: 0.4203
Epoch 7/10
38221/38221 [=====================

In [139]:
def get_model_txt_1():
    inp = Input(shape=(TXT_LEN,))
        
        
    x1 = Dense(512, activation='relu')(inp)
    x1 = Dropout(0.25)(x1)
    
    x2 = Dense(512, activation='relu')(x1)
    x2 = Dropout(0.5)(x2)
    
    x = concatenate([x1, x2])
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [140]:
model_txt_1 = get_model_txt_1()
model_txt_1.fit(x_txt_train, y_train, epochs=10, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/10
38221/38221 [==============================] - 19s 504us/sample - loss: 2.3747 - accuracy: 0.3932 - val_loss: 1.9832 - val_accuracy: 0.4827
Epoch 2/10
38221/38221 [==============================] - 19s 486us/sample - loss: 2.0407 - accuracy: 0.4706 - val_loss: 1.9111 - val_accuracy: 0.4926
Epoch 3/10
38221/38221 [==============================] - 20s 532us/sample - loss: 1.9414 - accuracy: 0.4901 - val_loss: 1.8837 - val_accuracy: 0.5015
Epoch 4/10
38221/38221 [==============================] - 20s 510us/sample - loss: 1.8688 - accuracy: 0.5062 - val_loss: 1.9051 - val_accuracy: 0.4933
Epoch 5/10
38221/38221 [==============================] - 21s 543us/sample - loss: 1.8086 - accuracy: 0.5162 - val_loss: 1.8612 - val_accuracy: 0.5069
Epoch 6/10
38221/38221 [==============================] - 23s 595us/sample - loss: 1.7497 - accuracy: 0.5283 - val_loss: 1.8907 - val_accuracy: 0.5077
Epoch 7/10
38221/38221 [=====================

In [7]:
def get_model_2():
    x_img = Dense(128, activation='relu')(inp_img)
    x_img = Dropout(0.25)(x_img)
    x_img = Dense(128, activation='tanh')(x_img)

    x_txt = Dense(128, activation='relu')(inp_txt)
    x_txt = Dropout(0.25)(x_txt)
    x_txt = Dense(128, activation='tanh')(x_txt)

    x = concatenate([x_img, x_txt])
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(256, activation='tanh')(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [172]:
model = get_model_2()
model.fit([x_img, x_txt], y, epochs=10, validation_split=0.1)

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 3s 344us/sample - loss: 3.0224 - accuracy: 0.2210 - categorical_accuracy: 0.2210 - val_loss: 2.2648 - val_accuracy: 0.3740 - val_categorical_accuracy: 0.3740
Epoch 2/10
9000/9000 [==============================] - 2s 179us/sample - loss: 2.1881 - accuracy: 0.4006 - categorical_accuracy: 0.4006 - val_loss: 2.0103 - val_accuracy: 0.4350 - val_categorical_accuracy: 0.4350
Epoch 3/10
9000/9000 [==============================] - 2s 180us/sample - loss: 1.9779 - accuracy: 0.4573 - categorical_accuracy: 0.4573 - val_loss: 1.8241 - val_accuracy: 0.5070 - val_categorical_accuracy: 0.5070
Epoch 4/10
9000/9000 [==============================] - 2s 177us/sample - loss: 1.8515 - accuracy: 0.4900 - categorical_accuracy: 0.4900 - val_loss: 1.8211 - val_accuracy: 0.4930 - val_categorical_accuracy: 0.4930
Epoch 5/10
9000/9000 [==============================] - 2s 176us/sample - loss: 1.7367 - accurac

In [115]:
model.evaluate([x_img_test, x_txt_test], y_test, verbose=0)

[0.04970714685320854, 0.98471624]

upgrades:  
1) bigger width of first layers  
2) add dropout before concatenation  
3) set dropout rate to 0.5  
4) only relu activations  
5) adadelt - decrease acc significantly  
6) like in resnet 2.0 use batch normalization (bet that will not help) . 


In [56]:
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.001, patience=3)

def get_model_3():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.5)(x_img)
    x_img = BatchNormalization()(x_img)
    x_img = Dense(512, activation='relu')(x_img)
    x_img = Dropout(0.5)(x_img)

    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.5)(x_txt)
    x_txt = BatchNormalization()(x_txt)
    x_txt = Dense(256, activation='relu')(x_txt)
    x_txt = Dropout(0.5)(x_txt)

    x = concatenate([x_img, x_txt])
    x = Dense(256, activation='relu')(x)
    relu = Dense(256, activation='relu')(x)
    sigmoid = Dense(256, activation='sigmoid')(x)
    mult_1 = Multiply()([x, sigmoid])
    minus = Lambda(lambda x: 1.0 - x)(sigmoid)
    mult_2 = Multiply()([minus, relu])
    x = Add()([mult_2, mult_1])

#     x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = concatenate([x_img, x_txt, x]) # smth residual
#     x = Dense(256, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

upgrades:  
1) remove early stopping  
2) epochs = 40

In [58]:
model_3 = get_model_3()
model_3.fit([x_img_train, x_txt_train], y_train, epochs=40, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/40
38221/38221 [==============================] - 60s 2ms/sample - loss: 2.1722 - accuracy: 0.4370 - val_loss: 1.6440 - val_accuracy: 0.5675
Epoch 2/40
38221/38221 [==============================] - 49s 1ms/sample - loss: 1.8382 - accuracy: 0.5159 - val_loss: 1.5833 - val_accuracy: 0.5861
Epoch 3/40
38221/38221 [==============================] - 49s 1ms/sample - loss: 1.7720 - accuracy: 0.5251 - val_loss: 1.5624 - val_accuracy: 0.5962
Epoch 4/40
38221/38221 [==============================] - 55s 1ms/sample - loss: 1.7325 - accuracy: 0.5375 - val_loss: 1.5649 - val_accuracy: 0.5889
Epoch 5/40
38221/38221 [==============================] - 48s 1ms/sample - loss: 1.7034 - accuracy: 0.5407 - val_loss: 1.5711 - val_accuracy: 0.5863
Epoch 6/40
38221/38221 [==============================] - 55s 1ms/sample - loss: 1.6841 - accuracy: 0.5425 - val_loss: 1.5485 - val_accuracy: 0.5934
Epoch 7/40
38221/38221 [==============================] -

KeyboardInterrupt: 

In [ ]:
model_3 = get_model_3()
model_3.fit([x_img_train, x_txt_train], y_train, epochs=40, validation_split=0.1)

In [62]:
def get_model_more_highways():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.5)(x_img)
    x_img = BatchNormalization()(x_img)
    x_img = Dense(512, activation='relu')(x_img)
    x_img = Dropout(0.5)(x_img)

    relu = Dense(512, activation='relu')(x_img)
    sigmoid = Dense(512, activation='sigmoid')(x_img)
    mult_1 = Multiply()([x_img, sigmoid])
    minus = Lambda(lambda x: 1.0 - x)(sigmoid)
    mult_2 = Multiply()([minus, relu])
    x_img = Add()([mult_2, mult_1])
    x_img = Dropout(0.5)(x_img)
                                         
    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.5)(x_txt)
    x_txt = BatchNormalization()(x_txt)
    x_txt = Dense(256, activation='relu')(x_txt)
    x_txt = Dropout(0.5)(x_txt)

    relu = Dense(256, activation='relu')(x_txt)
    sigmoid = Dense(256, activation='sigmoid')(x_txt)
    mult_1 = Multiply()([x_txt, sigmoid])
    minus = Lambda(lambda x: 1.0 - x)(sigmoid)
    mult_2 = Multiply()([minus, relu])
    x_txt = Add()([mult_2, mult_1])
    x_txt = Dropout(0.5)(x_txt)
                                         
    x = concatenate([x_img, x_txt])
    x = Dense(256, activation='relu')(x)
    relu = Dense(256, activation='relu')(x)
    sigmoid = Dense(256, activation='sigmoid')(x)
    mult_1 = Multiply()([x, sigmoid])
    minus = Lambda(lambda x: 1.0 - x)(sigmoid)
    mult_2 = Multiply()([minus, relu])
    x = Add()([mult_2, mult_1])

#     x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = concatenate([x_img, x_txt, x]) # smth residual
#     x = Dense(256, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [63]:
model_4 = get_model_more_highways()
model_4.fit([x_img_train, x_txt_train], y_train, epochs=40, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/40
38221/38221 [==============================] - 78s 2ms/sample - loss: 2.4668 - accuracy: 0.3681 - val_loss: 1.7522 - val_accuracy: 0.5477
Epoch 2/40
38221/38221 [==============================] - 69s 2ms/sample - loss: 2.0252 - accuracy: 0.4672 - val_loss: 1.6892 - val_accuracy: 0.5689
Epoch 3/40
38221/38221 [==============================] - 69s 2ms/sample - loss: 1.9499 - accuracy: 0.4912 - val_loss: 1.6681 - val_accuracy: 0.5698
Epoch 4/40
38221/38221 [==============================] - 70s 2ms/sample - loss: 1.8968 - accuracy: 0.5002 - val_loss: 1.6581 - val_accuracy: 0.5783
Epoch 5/40
38221/38221 [==============================] - 77s 2ms/sample - loss: 1.8602 - accuracy: 0.5067 - val_loss: 1.6336 - val_accuracy: 0.5781
Epoch 6/40
38221/38221 [==============================] - 74s 2ms/sample - loss: 1.8339 - accuracy: 0.5144 - val_loss: 1.6096 - val_accuracy: 0.5811
Epoch 7/40
38221/38221 [==============================] -

In [64]:
def get_model_5():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.5)(x_img)
    x_img = BatchNormalization()(x_img)
    x_img = Dense(512, activation='relu')(x_img)
    x_img = Dropout(0.5)(x_img)

    relu = Dense(512, activation='relu')(x_img)
    sigmoid = Dense(512, activation='sigmoid')(x_img)
    mult_1 = Multiply()([x_img, sigmoid])
    minus = Lambda(lambda x: 1.0 - x)(sigmoid)
    mult_2 = Multiply()([minus, relu])
    x_img = Add()([mult_2, mult_1])
    x_img = Dropout(0.5)(x_img)
                                         
    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.5)(x_txt)
    x_txt = BatchNormalization()(x_txt)
    x_txt = Dense(256, activation='relu')(x_txt)
    x_txt = Dropout(0.5)(x_txt)

    relu = Dense(256, activation='relu')(x_txt)
    sigmoid = Dense(256, activation='sigmoid')(x_txt)
    mult_1 = Multiply()([x_txt, sigmoid])
    minus = Lambda(lambda x: 1.0 - x)(sigmoid)
    mult_2 = Multiply()([minus, relu])
    x_txt = Add()([mult_2, mult_1])
    x_txt = Dropout(0.5)(x_txt)
                                         
    x = concatenate([x_img, x_txt])
    x = Dense(256, activation='relu')(x)
    relu = Dense(256, activation='relu')(x)
    sigmoid = Dense(256, activation='sigmoid')(x)
    mult_1 = Multiply()([x, sigmoid])
    minus = Lambda(lambda x: 1.0 - x)(sigmoid)
    mult_2 = Multiply()([minus, relu])
    x = Add()([mult_2, mult_1])

#     x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
#     x = Dense(256, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [65]:
model_5 = get_model_5()
model_5.fit([x_img_train, x_txt_train], y_train, epochs=30, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 53s 1ms/sample - loss: 2.8627 - accuracy: 0.2699 - val_loss: 2.0058 - val_accuracy: 0.4822
Epoch 2/30
38221/38221 [==============================] - 51s 1ms/sample - loss: 2.2518 - accuracy: 0.4131 - val_loss: 1.8349 - val_accuracy: 0.5147
Epoch 3/30
33568/38221 [=========================>....] - ETA: 6s - loss: 2.1166 - accuracy: 0.4468

KeyboardInterrupt: 

In [82]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import warnings

from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import backend as K
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.keras.utils.generic_utils import to_list
from  tensorflow.keras import regularizers 
from tensorflow.keras import constraints
from tensorflow.keras import activations
from tensorflow.keras import initializers

class Highway(Layer):
    """Densely connected highway network.
    Highway layers are a natural extension of LSTMs to feedforward networks.
    # Arguments
        init: name of initialization function for the weights of the layer
            (see [initializations](../initializations.md)),
            or alternatively, Theano function to use for weights
            initialization. This parameter is only relevant
            if you don't pass a `weights` argument.
        activation: name of activation function to use
            (see [activations](../activations.md)),
            or alternatively, elementwise Theano function.
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: a(x) = x).
        weights: list of Numpy arrays to set as initial weights.
            The list should have 2 elements, of shape `(input_dim, output_dim)`
            and (output_dim,) for weights and biases respectively.
        W_regularizer: instance of [WeightRegularizer](../regularizers.md)
            (eg. L1 or L2 regularization), applied to the main weights matrix.
        b_regularizer: instance of [WeightRegularizer](../regularizers.md),
            applied to the bias.
        activity_regularizer: instance of [ActivityRegularizer](../regularizers.md),
            applied to the network output.
        W_constraint: instance of the [constraints](../constraints.md) module
            (eg. maxnorm, nonneg), applied to the main weights matrix.
        b_constraint: instance of the [constraints](../constraints.md) module,
            applied to the bias.
        bias: whether to include a bias
            (i.e. make the layer affine rather than linear).
        input_dim: dimensionality of the input (integer). This argument
            (or alternatively, the keyword argument `input_shape`)
            is required when using this layer as the first layer in a model.
    # Input shape
        2D tensor with shape: `(nb_samples, input_dim)`.
    # Output shape
        2D tensor with shape: `(nb_samples, input_dim)`.
    # References
        - [Highway Networks](http://arxiv.org/abs/1505.00387v2)
    """

    def __init__(self,
                 init='glorot_uniform',
                 activation=None,
                 weights=None,
                 W_regularizer=None,
                 b_regularizer=None,
                 activity_regularizer=None,
                 W_constraint=None,
                 b_constraint=None,
                 bias=True,
                 input_dim=None,
                 **kwargs):
        
        if 'transform_bias' in kwargs:
            kwargs.pop('transform_bias')
            warnings.warn('`transform_bias` argument is deprecated and '
                          'has been removed.')
        self.init = initializers.get(init)
        self.activation = activations.get(activation)

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

        self.input_dim = input_dim
        if self.input_dim:
            kwargs['input_shape'] = (self.input_dim,)
        super(Highway, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(),
                                    shape=(None, input_dim))

        self.W = self.add_weight(shape=(input_dim, input_dim),
                                 initializer=self.init,
                                 name='W',
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.W_carry = self.add_weight(shape=(input_dim, input_dim),
                                       initializer=self.init,
                                       name='W_carry')
        if self.bias:
            self.b = self.add_weight(shape=(input_dim,),
                                     initializer='zero',
                                     name='b',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.b_carry = self.add_weight(shape=(input_dim,),
                                           initializer='one',
                                           name='b_carry')
        else:
            self.b_carry = None

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, x):
        y = K.dot(x, self.W_carry)
        if self.bias:
            y += self.b_carry
        transform_weight = activations.sigmoid(y)
        y = K.dot(x, self.W)
        if self.bias:
            y += self.b
        act = self.activation(y)
        act *= transform_weight
        output = act + (1 - transform_weight) * x
        return output

    def get_config(self):
        config = {'init': initializers.serialize(self.init),
                  'activation': activations.serialize(self.activation),
                  'W_regularizer': regularizers.serialize(self.W_regularizer),
                  'b_regularizer': regularizers.serialize(self.b_regularizer),
                  'activity_regularizer':
                      regularizers.serialize(self.activity_regularizer),
                  'W_constraint': constraints.serialize(self.W_constraint),
                  'b_constraint': constraints.serialize(self.b_constraint),
                  'bias': self.bias,
                  'input_dim': self.input_dim}
        base_config = super(Highway, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [83]:
def get_model_6():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.5)(x_img)
    x_img = BatchNormalization()(x_img)
    x_img = Dense(512, activation='relu')(x_img)
    x_img = Dropout(0.5)(x_img)

    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.5)(x_txt)
    x_txt = BatchNormalization()(x_txt)
    x_txt = Dense(256, activation='relu')(x_txt)
    x_txt = Dropout(0.5)(x_txt)

    x = concatenate([x_img, x_txt])
    x = Dense(256, activation='relu')(x)
    x = Highway()(x)
    
#     x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
#     x = concatenate([x_img, x_txt, x]) # smth residual
#     x = Dense(256, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [84]:
model_6 = get_model_6()
model_6.fit([x_img_train, x_txt_train], y_train, epochs=30, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 40s 1ms/sample - loss: 2.4450 - accuracy: 0.3659 - val_loss: 1.7573 - val_accuracy: 0.5361
Epoch 2/30
38221/38221 [==============================] - 40s 1ms/sample - loss: 1.9604 - accuracy: 0.4817 - val_loss: 1.6509 - val_accuracy: 0.5587
Epoch 3/30
38221/38221 [==============================] - 38s 1ms/sample - loss: 1.8686 - accuracy: 0.5052 - val_loss: 1.6237 - val_accuracy: 0.5679
Epoch 4/30
38221/38221 [==============================] - 30s 796us/sample - loss: 1.8087 - accuracy: 0.5197 - val_loss: 1.6133 - val_accuracy: 0.5757
Epoch 5/30
38221/38221 [==============================] - 28s 741us/sample - loss: 1.7872 - accuracy: 0.5227 - val_loss: 1.6092 - val_accuracy: 0.5809
Epoch 6/30
38221/38221 [==============================] - 29s 751us/sample - loss: 1.7718 - accuracy: 0.5294 - val_loss: 1.6202 - val_accuracy: 0.5759
Epoch 7/30
38221/38221 [===========================

In [87]:
def get_model_7():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.5)(x_img)
    x_img = Highway()(x_img)
    x_img = Dropout(0.5)(x_img)

    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.5)(x_txt)
    x_txt = Highway()(x_txt)
    x_txt = Dropout(0.5)(x_txt)

    x = concatenate([x_img, x_txt])
    x = Dense(256, activation='relu')(x)
    x = Highway()(x)
    
    x = Dropout(0.5)(x)
#     x = concatenate([x_img, x_txt, x]) # smth residual
#     x = Dense(256, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [88]:
model_7 = get_model_7()
model_7.fit([x_img_train, x_txt_train], y_train, epochs=30, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 93s 2ms/sample - loss: 2.4309 - accuracy: 0.3687 - val_loss: 1.7812 - val_accuracy: 0.5248
Epoch 2/30
38221/38221 [==============================] - 81s 2ms/sample - loss: 1.9300 - accuracy: 0.4918 - val_loss: 1.6566 - val_accuracy: 0.5595
Epoch 3/30
38221/38221 [==============================] - 79s 2ms/sample - loss: 1.8102 - accuracy: 0.5204 - val_loss: 1.6344 - val_accuracy: 0.5696
Epoch 4/30
38221/38221 [==============================] - 80s 2ms/sample - loss: 1.7490 - accuracy: 0.5376 - val_loss: 1.6181 - val_accuracy: 0.5722
Epoch 5/30
38221/38221 [==============================] - 84s 2ms/sample - loss: 1.7099 - accuracy: 0.5489 - val_loss: 1.5988 - val_accuracy: 0.5814
Epoch 6/30
38221/38221 [==============================] - 83s 2ms/sample - loss: 1.6748 - accuracy: 0.5556 - val_loss: 1.5945 - val_accuracy: 0.5776
Epoch 7/30
38221/38221 [==============================] -

KeyboardInterrupt: 

In [89]:
def get_model_8():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.5)(x_img)
    x_img = Highway()(x_img)
    x_img = Dropout(0.5)(x_img)

    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.5)(x_txt)
    x_txt = Highway()(x_txt)
    x_txt = Dropout(0.5)(x_txt)

    x = concatenate([x_img, x_txt])
    x = Dense(256, activation='relu')(x)
    x = Highway()(x)
    
    x = Dropout(0.5)(x)
    x = concatenate([x_img, x_txt, x]) # smth residual
    x = Dense(256, activation='relu')(x)
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [90]:
model_8 = get_model_8()
model_8.fit([x_img_train, x_txt_train], y_train, epochs=30, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 97s 3ms/sample - loss: 2.1876 - accuracy: 0.4245 - val_loss: 1.6591 - val_accuracy: 0.5571
Epoch 2/30
38221/38221 [==============================] - 102s 3ms/sample - loss: 1.7865 - accuracy: 0.5209 - val_loss: 1.6180 - val_accuracy: 0.5708
Epoch 3/30
38221/38221 [==============================] - 82s 2ms/sample - loss: 1.6994 - accuracy: 0.5427 - val_loss: 1.6130 - val_accuracy: 0.5792
Epoch 4/30
38221/38221 [==============================] - 97s 3ms/sample - loss: 1.6455 - accuracy: 0.5540 - val_loss: 1.5814 - val_accuracy: 0.5776
Epoch 5/30
38221/38221 [==============================] - 108s 3ms/sample - loss: 1.5968 - accuracy: 0.5660 - val_loss: 1.5665 - val_accuracy: 0.5825
Epoch 6/30
38221/38221 [==============================] - 107s 3ms/sample - loss: 1.5662 - accuracy: 0.5726 - val_loss: 1.5382 - val_accuracy: 0.5943
Epoch 7/30
38221/38221 [==============================

KeyboardInterrupt: 

In [95]:
def get_model_9():
    inp_img = Input(shape=(1024,))
    inp_txt = Input(shape=(300,))
    
    x_img = Dense(1024, activation='relu')(inp_img)
    x_img = Dropout(0.5)(x_img)
    x_img = Highway()(x_img)
    x_img = Dropout(0.5)(x_img)

    x_txt = Dense(512, activation='relu')(inp_txt)
    x_txt = Dropout(0.5)(x_txt)
    x_txt = Highway()(x_txt)
    x_txt = Dropout(0.5)(x_txt)

    x = concatenate([x_img, x_txt])
    x = Dense(256)(x)
    x = Highway()(x)
    x1 = Dropout(0.5)(x)
    
    x = concatenate([x_img, x_txt, x1]) # smth residual
    x = Dense(256)(x)
    x = Highway()(x)
    x = Dropout(0.5)(x)
    
    out = Dense(50, activation='softmax')(x)

    model = Model(inputs=[inp_img, inp_txt], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [96]:
model_9 = get_model_9()
model_9.fit([x_img_train, x_txt_train], y_train, epochs=30, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 63s 2ms/sample - loss: 2.4688 - accuracy: 0.3751 - val_loss: 1.9225 - val_accuracy: 0.5032
Epoch 2/30
38221/38221 [==============================] - 59s 2ms/sample - loss: 2.1493 - accuracy: 0.4545 - val_loss: 1.8679 - val_accuracy: 0.5166
Epoch 3/30
38221/38221 [==============================] - 58s 2ms/sample - loss: 2.0595 - accuracy: 0.4741 - val_loss: 1.8319 - val_accuracy: 0.5413
Epoch 4/30
38221/38221 [==============================] - 58s 2ms/sample - loss: 2.0058 - accuracy: 0.4850 - val_loss: 1.8237 - val_accuracy: 0.5286
Epoch 5/30
38221/38221 [==============================] - 58s 2ms/sample - loss: 1.9698 - accuracy: 0.4941 - val_loss: 1.7971 - val_accuracy: 0.5418
Epoch 6/30
38221/38221 [==============================] - 72s 2ms/sample - loss: 1.9207 - accuracy: 0.5018 - val_loss: 1.7599 - val_accuracy: 0.5493
Epoch 7/30
38221/38221 [==============================] -

In [102]:
print(x_img_test[6])

[0.01431737 0.02877704 0.16622615 ... 0.92601949 0.55370623 1.17222333]


In [110]:
x_img_train, x_img_test, x_txt_train, x_txt_test, y_train, y_test = train_test_split(norm_x_img_q, norm_x_txt_q, y_q, test_size=0.2, random_state=42)

In [111]:
model_3 = get_model_3()
model_3.fit([x_img_train, x_txt_train], y_train, epochs=30, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 36s 933us/sample - loss: 2.1534 - accuracy: 0.4437 - val_loss: 1.6515 - val_accuracy: 0.5637
Epoch 2/30
38221/38221 [==============================] - 33s 863us/sample - loss: 1.8393 - accuracy: 0.5168 - val_loss: 1.5814 - val_accuracy: 0.5877
Epoch 3/30
38221/38221 [==============================] - 32s 828us/sample - loss: 1.7804 - accuracy: 0.5280 - val_loss: 1.5548 - val_accuracy: 0.5854
Epoch 4/30
38221/38221 [==============================] - 32s 847us/sample - loss: 1.7407 - accuracy: 0.5386 - val_loss: 1.5312 - val_accuracy: 0.5943
Epoch 5/30
38221/38221 [==============================] - 31s 812us/sample - loss: 1.7070 - accuracy: 0.5442 - val_loss: 1.5338 - val_accuracy: 0.5992
Epoch 6/30
38221/38221 [==============================] - 31s 815us/sample - loss: 1.6805 - accuracy: 0.5546 - val_loss: 1.5188 - val_accuracy: 0.6028
Epoch 7/30
38221/38221 [=====================